In [10]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from math import factorial
import numpy as np
from scipy import stats as st

In [11]:
visitas = pd.read_csv('datos/visits_log_us.csv')

In [12]:
pedidos = pd.read_csv('datos/orders_log_us.csv')

In [13]:
gastos = pd.read_csv('datos/costs_us.csv')

VISITAS 

In [14]:
visitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


In [6]:
visitas.head()

,Device,End Ts,Source Id,Start Ts,Uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


In [7]:
visitas = visitas.isnull().mean() * 100
print(visitas)

Device       0.0
End Ts       0.0
Source Id    0.0
Start Ts     0.0
Uid          0.0
dtype: float64


In [15]:
visitas.duplicated().sum()

np.int64(0)

In [16]:
visitas = visitas.drop_duplicates()
print(visitas.duplicated().sum())
visitas.reset_index(drop=True)

0


,Device,End Ts,Source Id,Start Ts,Uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168
...,...,...,...,...,...
359395,desktop,2017-07-29 19:07:19,2,2017-07-29 19:07:00,18363291481961487539
359396,touch,2018-01-25 17:38:19,1,2018-01-25 17:38:00,18370831553019119586
359397,desktop,2018-03-03 10:12:19,4,2018-03-03 10:12:00,18387297585500748294
359398,desktop,2017-11-02 10:12:19,5,2017-11-02 10:12:00,18388616944624776485


Convertiremos las columnas 'Start Ts' y 'End Ts' a formato datetime para poder trabajar con ellas

In [17]:
visitas['Start Ts'] = pd.to_datetime(visitas['Start Ts'])
visitas['End Ts'] = pd.to_datetime(visitas['End Ts'])

Ahora Trabajaremos para encontrar la cantidad de usuarios por día, semana, mes

In [18]:
u_dia = visitas.groupby(visitas['Start Ts'].dt.date)['Uid'].nunique()
u_semana = visitas.groupby(visitas['Start Ts'].dt.to_period('W'))['Uid'].nunique()
u_mes = visitas.groupby(visitas['Start Ts'].dt.to_period('M'))['Uid'].nunique()

print(u_dia.head(5))
print(u_semana.head(5))
print(u_mes.head(5))

Start Ts
2017-06-01    605
2017-06-02    608
2017-06-03    445
2017-06-04    476
2017-06-05    820
Name: Uid, dtype: int64
Start Ts
2017-05-29/2017-06-04    2021
2017-06-05/2017-06-11    4129
2017-06-12/2017-06-18    2812
2017-06-19/2017-06-25    2878
2017-06-26/2017-07-02    3064
Freq: W-SUN, Name: Uid, dtype: int64
Start Ts
2017-06    13259
2017-07    14183
2017-08    11631
2017-09    18975
2017-10    29692
Freq: M, Name: Uid, dtype: int64


Revisaremos cuantas sesiones se realizan por día.

In [19]:
ses_por_dia = visitas.groupby(visitas['Start Ts'].dt.date)['Uid'].count()
print("Sesiones por dia:\n", ses_por_dia.head(5))

Sesiones por dia:
 Start Ts
2017-06-01    664
2017-06-02    658
2017-06-03    477
2017-06-04    510
2017-06-05    893
Name: Uid, dtype: int64


Revisaremos si un usuario ingresa más de una vez

In [20]:
ses_por_us = visitas.groupby('Uid')['Start Ts'].count()
multiple_ses_us = ses_por_us[ses_por_us > 1]

print("Usuarios con más de una Sesión por Día:\n", multiple_ses_us)

Usuarios con más de una Sesión por Día:
 Uid
313578113262317         3
325320750514679         2
526778907996220         4
577434573913691         2
1260655184775459        4
                       ..
18445884613277162497    3
18446104389491815722    2
18446156210226471712    7
18446167067214817906    2
18446621818809592527    2
Name: Start Ts, Length: 52128, dtype: int64


Vamos a revisar cuanto dura cada sesión del visitante

In [21]:
visitas['duracion'] = visitas['End Ts'] - visitas['Start Ts']
print("Duracion de Cada Sesion:\n", visitas['duracion'])

Duracion de Cada Sesion:
 0        0 days 00:18:00
1        0 days 00:28:00
2        0 days 00:00:00
3        0 days 00:24:00
4        0 days 00:00:00
               ...      
359395   0 days 00:00:19
359396   0 days 00:00:19
359397   0 days 00:00:19
359398   0 days 00:00:19
359399   0 days 00:00:19
Name: duracion, Length: 359400, dtype: timedelta64[ns]


Revisaremos con que frecuencia regresa al sitio cada usuario

In [22]:
visitas['Start Ts'] = pd.to_datetime(visitas['Start Ts'])
visitas = visitas.sort_values(by=['Uid','Start Ts'])
frecuencia_regreso = visitas.groupby('Uid')['Start Ts'].diff()
print(frecuencia_regreso)

196543                 NaT
257333                 NaT
173818                 NaT
173142                 NaT
192554   152 days 16:28:00
                ...       
175259                 NaT
293449                 NaT
319162                 NaT
222173   144 days 20:03:00
132805                 NaT
Name: Start Ts, Length: 359400, dtype: timedelta64[ns]
